In [1]:
!pip install sportsdataverse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.9/207.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.9.0
    Uninstalling polars-1.9.0:
      Successfully uninstalled polars-1.9.0


In [2]:
!pip install great_tables

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.2/607.2 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
from great_tables import GT, html, style, loc
import sportsdataverse as sdv

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [16]:
players_df = pd.read_parquet("https://raw.githubusercontent.com/erikhall6373/big_data_bowl_2025/refs/heads/main/data/players.parquet")

In [50]:
player_separation_df = pd.read_csv("https://raw.githubusercontent.com/erikhall6373/big_data_bowl_2025/refs/heads/main/modeling/separationaddedplayer.csv")
defender_separation_df = pd.read_csv("https://raw.githubusercontent.com/erikhall6373/big_data_bowl_2025/refs/heads/main/modeling/separationaddeddefender.csv")
team_separation_df = pd.read_csv("https://raw.githubusercontent.com/erikhall6373/big_data_bowl_2025/refs/heads/main/modeling/separationaddedteam.csv")

In [21]:
roster_df = sdv.nfl_loaders.load_nfl_rosters(seasons = [2022], return_as_pandas = True)

100%|██████████| 1/1 [00:00<00:00,  2.99it/s]


In [35]:
roster_headshots_df = roster_df[["gsis_it_id", "full_name", "headshot_url"]]
roster_headshots_df = roster_headshots_df.rename(columns = {"gsis_it_id": "nflId"})
roster_headshots_df["nflId"] = roster_headshots_df["nflId"].astype(int)

In [52]:
def build_defender_summary_df():

  defender_separation_top_10_df = defender_separation_df.sort_values(by = "postSnapSeparationAdded", ascending = True).head(10)
  defender_separation_top_10_df["nflId"] = defender_separation_top_10_df["defender_nflId"].astype(int)
  defender_separation_top_10_df = defender_separation_top_10_df[["nflId", "predictedMotionSeparation", "postSnapSeparationAdded",
                                                             "percentRoutesWithAddedPostSnapSeparation", "count"]]

  defender_separation_top_10_df = defender_separation_top_10_df.merge(roster_headshots_df, on = "nflId", how = "left")

  return defender_separation_top_10_df

In [53]:
def build_player_summary_df():

  player_separation_top_10_df = player_separation_df.sort_values(by = "postSnapSeparationAdded", ascending = False).head(10)
  player_separation_top_10_df["nflId"] = player_separation_top_10_df["nflId"].astype(int)
  player_separation_top_10_df = player_separation_top_10_df[["nflId", "predictedMotionSeparation", "postSnapSeparationAdded",
                                                             "percentRoutesWithAddedPostSnapSeparation", "count"]]

  player_separation_top_10_df = player_separation_top_10_df.merge(roster_headshots_df, on = "nflId", how = "left")

  return player_separation_top_10_df

In [56]:
def create_headshot_html(summary_df):

  result_df = summary_df.copy()
  result_df["headshot_html"] = result_df['headshot_url'].apply(lambda row: f'<img src="{row}" height = 40;/>')

  return result_df

In [57]:
def font_538_formatting(df, cols):

  google_font = '<link href="https://fonts.googleapis.com/css?family=Chivo" rel="stylesheet">'

  html_pre = f"{google_font}<p style='font-family: Chivo; bottom: 0; font-size: 14px; weight: 200'> "
  html_post = "</p>"

  result_df = df.copy()

  for col in cols:
    result_df[col] = result_df[col].apply(lambda row: f"{html_pre}{row}{html_post}")

  return result_df

In [58]:
def formatting_538(great_table_obj):
  great_table_obj = great_table_obj.tab_options(column_labels_background_color = "white",
                          data_row_padding = "3px",
                          heading_border_bottom_style = None,
                          table_border_top_width = "3px",
                          table_border_top_style = None,
                          table_border_bottom_style = None,
                          column_labels_font_weight = "normal",
                          column_labels_border_top_style = None,
                          column_labels_border_bottom_width = "2px",
                          column_labels_border_bottom_color = "black",
                          row_group_border_top_style = None,
                          row_group_border_top_color = "black",
                          row_group_border_bottom_width = "1px",
                          row_group_border_bottom_color = "white",
                          stub_border_color = "white",
                          stub_border_width = "px",
                          source_notes_font_size = 12,
                          source_notes_border_lr_style = None,
                          table_font_size = 16,
                          heading_align = "left")

  return great_table_obj

In [59]:
def hulk_color_formatting(cols, table, green_good = False, trim = False):

  color_palette = ["#1b7837", "#7fbf7b", "#d9f0d3", "#f7f7f7", "#e7d4e8", "#af8dc3", "#762a83"]

  if green_good:
    color_palette = ["#762a83", "#af8dc3", "#e7d4e8", "#f7f7f7", "#d9f0d3", "#7fbf7b", "#1b7837"]

  if trim:
    color_palette = color_palette[1:len(color_palette) - 1]


  result_table = table.data_color(columns = cols, palette = color_palette)

  return result_table

In [89]:
def build_summary_gt(summary_df, player_col, player_name_col, analysis_cols_dict, rank_col_dict):

  summary_df = create_headshot_html(summary_df)
  summary_df = font_538_formatting(summary_df, player_name_col)

  analysis_col_names = list(analysis_cols_dict.keys())
  analysis_cols_green_good = [analysis_col for analysis_col in list(analysis_cols_dict.keys()) if analysis_cols_dict[analysis_col] == 'good']
  analysis_cols_green_bad = [analysis_col for analysis_col in list(analysis_cols_dict.keys()) if analysis_cols_dict[analysis_col] == 'bad']

  rank_col_name = list(rank_col_dict.keys())[0]
  rank_col_list = [rank_col_name]


  order_cols = player_name_col + player_col + analysis_col_names + rank_col_list

  #viz_df = font_538_formatting(viz_df, analysis_col_names)
  result_table = GT(summary_df[order_cols])
  result_table = formatting_538(result_table)
  result_table = hulk_color_formatting(analysis_cols_green_good, result_table, green_good = True, trim = True)
  result_table = hulk_color_formatting(analysis_cols_green_bad, result_table, trim = True)
  result_table = hulk_color_formatting(rank_col_list, result_table, green_good = rank_col_dict[rank_col_name] == "good")
  result_table = result_table.fmt_markdown(columns = player_col + player_name_col)

  return result_table

In [90]:
def cleanup_gt_formatting(summary_gt_object):

  result_gt = summary_gt_object.fmt_number(columns=["percentRoutesWithAddedPostSnapSeparation", "postSnapSeparationAdded"], decimals = 2)

  result_gt = result_gt.cols_label(full_name = 'Player', headshot_html = "",
                                   percentRoutesWithAddedPostSnapSeparation = "Percent of Routes with Separation Added",
                                   count = "Number of Plays",
                                   postSnapSeparationAdded = 'Post Snap Separation Added',
                                   )

  return result_gt

In [93]:
player_summary_gt = build_summary_gt(summary_df = build_player_summary_df(),
                        player_col = ["headshot_html"],
                        player_name_col = ["full_name"],
                        analysis_cols_dict = {"percentRoutesWithAddedPostSnapSeparation" : "good", "count" : "good"},
                        rank_col_dict = {"postSnapSeparationAdded" : "good"})

player_summary_gt = cleanup_gt_formatting(player_summary_gt)

In [94]:
defender_summary_gt = build_summary_gt(summary_df = build_defender_summary_df(),
                        player_col = ["headshot_html"],
                        player_name_col = ["full_name"],
                        analysis_cols_dict = {"percentRoutesWithAddedPostSnapSeparation" : "good", "count" : "good"},
                        rank_col_dict = {"postSnapSeparationAdded" : "good"})

defender_summary_gt = cleanup_gt_formatting(defender_summary_gt)

In [95]:
player_summary_gt

Player,,Percent of Routes with Separation Added,Number of Plays,Post Snap Separation Added
Kenneth Gainwell,,0.71,41,1.13
Devin Singletary,,0.62,100,1.07
Darrell Henderson,,0.66,88,1.07
Cordarrelle Patterson,,0.58,26,0.99
Connor Heyward,,0.56,34,0.99
DeeJay Dallas,,0.44,27,0.96
Parker Hesse,,0.53,55,0.96
Giovanni Ricci,,0.62,34,0.95
Brandon Powell,,0.41,29,0.95
Jaylen Warren,,0.51,53,0.79


In [96]:
defender_summary_gt

Player,,Percent of Routes with Separation Added,Number of Plays,Post Snap Separation Added
Andrew Adams,,0.28,46,−1.15
Devin Bush,,0.25,113,−1.11
Ryan Neal,,0.32,106,−0.94
Jamin Davis,,0.30,109,−0.88
Nasir Adderley,,0.32,93,−0.87
Justin Simmons,,0.30,66,−0.75
Rodney McLeod,,0.30,106,−0.75
Drue Tranquill,,0.31,124,−0.74
Marcus Jones,,0.24,45,−0.73
Kamren Curl,,0.32,110,−0.69
